In [329]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [330]:
podatki_iz_turnirjev = pd.read_csv("podatki_iz_turnirjev.csv")
podatki_o_narodnosti = pd.read_csv("podatki_o_narodnosti.csv")
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(podatki_iz_turnirjev["tip_zmage"])

In [331]:
podatki_iz_turnirjev['igralec_crni_rang'] = podatki_iz_turnirjev['igralec_crni_rang'].fillna(0)
podatki_iz_turnirjev['igralec_beli_rang'] = podatki_iz_turnirjev['igralec_beli_rang'].fillna(0)
podatki_iz_turnirjev['tip_zmage'] = podatki_iz_turnirjev['tip_zmage'].fillna(0)
podatki_iz_turnirjev.leto = podatki_iz_turnirjev.leto.astype(int)
podatki_iz_turnirjev.mesec = podatki_iz_turnirjev.mesec.astype(int)
podatki_iz_turnirjev.dan = podatki_iz_turnirjev.dan.astype(int)
podatki_iz_turnirjev.igralec_beli_rang = podatki_iz_turnirjev.igralec_beli_rang.astype(int)
podatki_iz_turnirjev.igralec_crni_rang = podatki_iz_turnirjev.igralec_crni_rang.astype(int)

In [332]:
def dobi_leto(leto, podatki):
    relevantni_podatki = podatki[podatki.leto == leto]
    return relevantni_podatki

def izberi_polja(polja, podatki):
    return podatki[polja]

def dobi_stevilo_iger(podatki):
    return podatki.shape[0]

In [333]:
bela_po_letih = []
crna_po_letih = []
for leto in range(1969, 2023):
    podatki = dobi_leto(leto, podatki_iz_turnirjev)
    podatki = podatki[podatki.zmagovalec != "Void"]
    podatki = izberi_polja(["zmagovalec"], podatki)
    stevilo_iger = dobi_stevilo_iger(podatki)
    podatki = podatki[podatki.zmagovalec == "W"]
    procent = dobi_stevilo_iger(podatki) / stevilo_iger
    bela_po_letih.append(procent)
    crna_po_letih.append(1 - procent)

In [334]:
verjetnost = []
for leto in range(1969, 2023):
    podatki = dobi_leto(leto, podatki_iz_turnirjev)
    stevilo_iger = podatki[(podatki.igralec_crni_rang > podatki.igralec_beli_rang) | (podatki.igralec_crni_rang < podatki.igralec_beli_rang)].size
    podatki = izberi_polja(["igralec_crni_rang", "igralec_beli_rang", "zmagovalec"], podatki)
    verjetnost.append(podatki[
        (podatki.igralec_crni_rang > podatki.igralec_beli_rang) & (podatki.zmagovalec == "W")
        |
        (podatki.igralec_crni_rang < podatki.igralec_beli_rang) & (podatki.zmagovalec == "B")].size / stevilo_iger)

In [335]:
drzave_po_letih = [{"Kitajska" : [0, 0], "Japonska" : [0, 0], "Koreja" : [0, 0], "Tajvan" : [0, 0], "Ostali" : [0, 0], "AI" : [0, 0]} for _  in range(54)]
for leto in range(1969, 2023):
    podatki = dobi_leto(leto, podatki_iz_turnirjev)
    podatki = izberi_polja(["igralec_crni", "igralec_beli", "zmagovalec"], podatki)
    podatki = podatki[podatki.zmagovalec != "Void"]
    for ind in podatki.index:
        if podatki["zmagovalec"][ind] == "W":
            igralec_z = podatki["igralec_beli"][ind].removesuffix("NR").removesuffix("Ama").removesuffix("Sr").removesuffix("Insei").strip()
            igralec_p = podatki["igralec_crni"][ind].removesuffix("NR").removesuffix("Ama").removesuffix("Sr").removesuffix("Insei").strip()
        else:
            igralec_z = podatki["igralec_crni"][ind].removesuffix("NR").removesuffix("Ama").removesuffix("Sr").removesuffix("Insei").strip()
            igralec_p = podatki["igralec_beli"][ind].removesuffix("NR").removesuffix("Ama").removesuffix("Sr").removesuffix("Insei").strip()
        drzava_z = podatki_o_narodnosti[podatki_o_narodnosti["Igralec"] == igralec_z].iat[0, 0]
        drzava_p = podatki_o_narodnosti[podatki_o_narodnosti["Igralec"] == igralec_p].iat[0, 0]
        if drzava_p != drzava_z:
            drzave_po_letih[leto - 1969][drzava_z][0] += 1
            drzave_po_letih[leto - 1969][drzava_z][1] += 1
            drzave_po_letih[leto - 1969][drzava_p][1] += 1

In [336]:
drzave_cez_leta = {"Kitajska" : [], "Japonska" : [], "Koreja" : [], "Tajvan" : [], "Ostali" : [], "AI" : []}
for leto in drzave_po_letih:
    for drzava in leto.keys():
        if leto[drzava][1] != 0:
            drzave_cez_leta[drzava].append(leto[drzava][0] / leto[drzava][1])
        else:
            drzave_cez_leta[drzava].append(0)

In [337]:
podatki = izberi_polja(["igralec_crni", "igralec_beli", "zmagovalec"], podatki_iz_turnirjev)
rezultati_igralcev = {}
podatki = podatki[podatki.zmagovalec != "Void"]
for ind in podatki.index:
    if podatki["zmagovalec"][ind] == "W":
        igralec_z = podatki["igralec_beli"][ind].removesuffix("NR").removesuffix("Ama").removesuffix("Sr").removesuffix("Insei").strip()
        igralec_p = podatki["igralec_crni"][ind].removesuffix("NR").removesuffix("Ama").removesuffix("Sr").removesuffix("Insei").strip()  
    else:
        igralec_z = podatki["igralec_crni"][ind].removesuffix("NR").removesuffix("Ama").removesuffix("Sr").removesuffix("Insei").strip()
        igralec_p = podatki["igralec_beli"][ind].removesuffix("NR").removesuffix("Ama").removesuffix("Sr").removesuffix("Insei").strip()
    if igralec_z in rezultati_igralcev.keys():
        rezultati_igralcev[igralec_z] = [rezultati_igralcev[igralec_z][0] + 1, rezultati_igralcev[igralec_z][1] + 1]
    else:
        rezultati_igralcev[igralec_z] = [1, 1]
    if igralec_p in rezultati_igralcev.keys():
        rezultati_igralcev[igralec_p] = [rezultati_igralcev[igralec_p][0], rezultati_igralcev[igralec_p][1] + 1]
    else:
        rezultati_igralcev[igralec_p] = [0, 1]

In [ ]:
wr_igralcev = {}
for igralec in rezultati_igralcev.keys():
    if rezultati_igralcev[igralec][1] >= 50:
        wr_igralcev[igralec] = rezultati_igralcev[igralec][0] / rezultati_igralcev[igralec][1]
sorted(wr_igralcev, key = wr_igralcev.get, reverse = True)[:20]

In [504]:
max_lenght = 0
data = podatki_o_narodnosti[(podatki_o_narodnosti.Drzava != "Ostali") & (podatki_o_narodnosti.Drzava != "AI")]
num_names = 0
for ind in data.index:
    num_names += 1
    name_lenght = len(data["Igralec"][ind])
    max_lenght = max_lenght if max_lenght > name_lenght else name_lenght

index_dict = {"Kitajska" : 0, "Japonska" : 1, "Koreja" : 2, "Tajvan" : 3}

encoded_names = []
for ind in data.index:
    encoded_name = np.zeros(max_lenght + 1)
    encoded_name[0] = int(index_dict[data["Drzava"][ind]])
    for letter in range(0, len(data["Igralec"][ind])):
        encoded_name[letter + 1] = ord(data["Igralec"][ind][letter]) / 127
    encoded_names.append(encoded_name)
encoded_names = np.asarray(encoded_names)

In [518]:
np.random.shuffle(encoded_names)
data_dev = encoded_names[0:210].T
Y_dev = data_dev[0]
X_dev = data_dev[1:]

data_train = encoded_names[210:].T
Y_train = data_train[0].astype(int)
X_train = data_train[1:]

In [519]:
def init_params():
    W1 = np.random.rand(10, 20) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    W3 = np.random.rand(4, 10) - 0.5
    b3 = np.random.rand(4, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

def ReLU(Z):
    return np.maximum(0, Z)

def deriv_ReLU(Z):
    return Z > 0

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backprop(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)

    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3)

    dZ2 = W3.T.dot(dZ3) * deriv_ReLU(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)

    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3
    return W1, b1, W2, b2, W3, b3

In [520]:
def get_predictions(A):
    return np.argmax(A, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backprop(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2, W3, b3

def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
    predictions = get_predictions(A3)
    return A3

In [544]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, iterations = 500, alpha = 0.001)

Iteration:  0
Accuracy:  0.00105318588730911
Iteration:  10
Accuracy:  0.00105318588730911
Iteration:  20
Accuracy:  0.00105318588730911
Iteration:  30
Accuracy:  0.00105318588730911
Iteration:  40
Accuracy:  0.001579778830963665
Iteration:  50
Accuracy:  0.001579778830963665
Iteration:  60
Accuracy:  0.001579778830963665
Iteration:  70
Accuracy:  0.00210637177461822
Iteration:  80
Accuracy:  0.00315955766192733
Iteration:  90
Accuracy:  0.006845708267509215
Iteration:  100
Accuracy:  0.006845708267509215
Iteration:  110
Accuracy:  0.008952080042127435
Iteration:  120
Accuracy:  0.013164823591363875
Iteration:  130
Accuracy:  0.015271195365982097
Iteration:  140
Accuracy:  0.019483938915218536
Iteration:  150
Accuracy:  0.02317008952080042
Iteration:  160
Accuracy:  0.02632964718272775
Iteration:  170
Accuracy:  0.031068983675618746
Iteration:  180
Accuracy:  0.037914691943127965
Iteration:  190
Accuracy:  0.04160084254870985
Iteration:  200
Accuracy:  0.04476040021063718
Iteration:  2

C:\Users\matev\AppData\Local\Temp\ipykernel_12656\267321682.py:17: RuntimeWarning: overflow encountered in exp
  return np.exp(Z) / np.sum(np.exp(Z))
C:\Users\matev\AppData\Local\Temp\ipykernel_12656\267321682.py:17: RuntimeWarning: invalid value encountered in divide
  return np.exp(Z) / np.sum(np.exp(Z))


In [543]:
encoded_name = np.zeros(max_lenght)
name = "Shin Jinseo"
for letter in range(0, len(name)):
    encoded_name[letter] = ord(name[letter]) / 127
make_predictions(np.asarray([encoded_name]).T, W1, b1, W2, b2, W3, b3)

array([[0.23939905],
       [0.32182904],
       [0.31511617],
       [0.12365574]])